In [7]:
import cv2
import numpy as np

In [8]:
# Load YOLOv3 model
net = cv2.dnn.readNet("darknet\cfg\yolov3.weights", "darknet\cfg\yolov3.cfg")

In [9]:
# Load COCO class names
with open("darknet\data\coco.names", "r") as f:
    classes = f.read().strip().split("\n")

In [10]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model

# Load pre-trained ResNet50 model without the classification head
base_model = ResNet50(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)

def extract_features(person_roi):
    # Resize the person_roi to match the input size of the model
    person_roi = cv2.resize(person_roi, (224, 224))
    person_roi = preprocess_input(person_roi)  # Preprocess input as required by ResNet50
    person_roi = np.expand_dims(person_roi, axis=0)  # Add batch dimension
    
    # Extract features using the pre-trained ResNet50 model
    features = model.predict(person_roi)
    
    # Flatten the features if needed
    features = features.flatten()
    
    return features

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

# Database containing features and person IDs
database = []

def reidentify_person(query_features):
    best_match_person_id = None
    best_similarity = -1  # Initialize with a low value
    
    for person_id, stored_features in database:
        similarity = cosine_similarity([query_features], [stored_features])[0][0]
        if similarity > best_similarity:
            best_similarity = similarity
            best_match_person_id = person_id
    
    # You can set a threshold to determine if it's a valid match
    if best_similarity > 0.7:
        return best_match_person_id
    else:
        return "Unknown"  # Return "Unknown" if no good match is found

In [12]:
# Set the confidence threshold for detections
# confidence_threshold = 0.5

# Initialize video capture (change 0 to your video file path if needed)
cap = cv2.VideoCapture('cctvfootage.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform person detection using YOLO
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(net.getUnconnectedOutLayersNames())

    # Process YOLO output for persons
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if classes[class_id] == "person" and confidence > 0.5:
                # Extract features from the person bounding box
                x, y, w, h = detection[0:4] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                person_roi = frame[int(y-h/2):int(y+h/2), int(x-w/2):int(x+w/2)]
                # Extract features from person_roi using a CNN
                features = extract_features(person_roi)
                # Compare features with the database and perform reidentification
                person_id = reidentify_person(features)

                # Draw bounding box and person ID on the frame
                cv2.rectangle(frame, (int(x-w/2), int(y-h/2)), (int(x+w/2), int(y+h/2)), (0, 255, 0), 2)
                cv2.putText(frame, f"Person {person_id}", (int(x-w/2), int(y-h/2)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow("CCTV Footage", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 158ms/step
